In [21]:
## import (and install if necessary) pandas
%pip install pandas
import pandas as pd
from datetime import datetime


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df = pd.read_csv("full_data.csv"
                 ,parse_dates=['TripDate','StartDateTime','EndDateTime','CustomerBirthDate']
                 )
## process cost monetary data
df['Cost'] = df['Cost'].str.replace(',', '')
df['Cost'] = df['Cost'].str.replace('$', '')
df['Cost'] = df['Cost'].str.replace('(', '')
df['Cost'] = df['Cost'].str.replace(')', '')
df['Cost'] = df['Cost'].astype(float)
print(df.info())
print(df.head(5))


In [23]:
## summary counts
runActualID_counts = df.groupby(['RunActualID'])['RunActualID'].count()
print(f"Maximum Number of rows per RunActualID: {runActualID_counts.max()}")
runActualID_vehicleID = df.groupby('RunActualID')['VehicleID'].nunique()
runActualIDs_more_than_one_vehicleID = runActualID_vehicleID[runActualID_vehicleID != 1]
print(f"Any RunActualIDs that have more than one VehicleID: {runActualIDs_more_than_one_vehicleID.nunique()}")
print(f"Unique number of RunActualIDs: {df['RunActualID'].nunique()}")
print(f"Unique number of CustomerIDs: {df['CustomerID'].nunique()}")
print(f"Unique number of VehicleIDs: {df['VehicleID'].nunique()}")

Maximum Number of rows per RunActualID: 18
Any RunActualIDs that have more than one VehicleID: 0
Unique number of RunActualIDs: 213272
Unique number of CustomerIDs: 10215
Unique number of VehicleIDs: 538


In [24]:
dates_per_runactualIDs = df.groupby(['RunActualID']).agg({'EndDateTime': 'max', 'StartDateTime':'min'})
dates_per_runactualIDs['EndDate'] = pd.to_datetime(dates_per_runactualIDs['EndDateTime'].dt.date)
dates_per_runactualIDs['StartDate'] = pd.to_datetime(dates_per_runactualIDs['StartDateTime'].dt.date)
dates_per_runactualIDs['diff'] = (dates_per_runactualIDs['EndDateTime'] - dates_per_runactualIDs['StartDateTime']).dt.total_seconds()
seconds_in_day = 86400
runactualIDs_over_day = dates_per_runactualIDs[dates_per_runactualIDs['diff'] > seconds_in_day]
#print(runactualIDs_over_day['RunActualId'].nunique())

print(f"""Number of RunActualIDs lasting over 24 hours: {runactualIDs_over_day.index.nunique()}
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv""")
runactualIDs_over_day.to_csv("runactualIDs_over_day.csv", columns = ['StartDateTime','EndDateTime'])

Number of RunActualIDs lasting over 24 hours: 4
(Calculated by subtracting EndDateTime from StartDateTime)
Saving outlier RunActualIDs to `runactualIDs_over_day.csv


In [79]:
df_route_fields = df[['RunActualID','CustomerID','StartDateTime','PickupX','PickupY','EndDateTime','DropoffX','DropoffY','Cost','CollectedFareType']]
df_route_fields = df_route_fields.sort_values(by=['RunActualID','StartDateTime'])
df_route_fields = df_route_fields.reset_index()
#print(df_route_fields)
g = {lambda x: x.head(1)
     ,lambda x: x.head(2)
    }
#df_routes = df_route_fields.groupby('RunActualID')[['CustomerID','StartDateTime']].apply(set)
#df_routes = df_route_fields.groupby('RunActualID')[['CustomerID','StartDateTime']].agg()
df_routes = df_route_fields.groupby('RunActualID').agg(
    first_customerID=pd.NamedAgg(column='CustomerID', aggfunc = lambda t: t.iloc[1]),
    second_customerID=pd.NamedAgg(column='CustomerID', aggfunc = lambda t: t.head[2])

    )
df_routes.to_csv("testing.csv")
print(df_routes)

IndexError: single positional indexer is out-of-bounds